In [1]:
import os
import pandas as pd

In [2]:
os.getcwd()

'd:\\Studies\\Materials\\Second-cycle\\I year\\II trimester\\ML and StatL\\ML\\ML-project\\src'

In [3]:
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')
mushrooms.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,...,s,y,w,u,w,t,g,NaN,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,...,s,y,w,u,w,t,g,NaN,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,...,s,y,w,u,w,t,g,NaN,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,...,s,y,w,u,w,t,p,NaN,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,...,s,y,w,u,w,t,p,NaN,d,w


In [4]:
mushrooms.dtypes

class                    object
cap-diameter            float64
cap-shape                object
cap-surface              object
cap-color                object
does-bruise-or-bleed     object
gill-attachment          object
gill-spacing             object
gill-color               object
stem-height             float64
stem-width              float64
stem-root                object
stem-surface             object
stem-color               object
veil-type                object
veil-color               object
has-ring                 object
ring-type                object
spore-print-color        object
habitat                  object
season                   object
dtype: object

In [5]:
n_rows = mushrooms.shape[0]
n_columns = mushrooms.shape[1]

print(f'There are {n_rows} rows and {n_columns} columns.')

There are 61069 rows and 21 columns.


In [6]:
print("There are no missing values." if not mushrooms.isnull().any().any()\
      else "There are missing values in the dataframe.")

There are missing values in the dataframe.


In [7]:
nan_counts = mushrooms.isna().sum()
nan_counts[nan_counts > 0]

cap-surface          14120
gill-attachment       9884
gill-spacing         25063
stem-root            51538
stem-surface         38124
veil-type            57892
veil-color           53656
ring-type             2471
spore-print-color    54715
dtype: int64

In [8]:
edible_count = (mushrooms['class'] == 'e').sum()
poisonous_count = mushrooms.shape[0] - edible_count
print(f"Number of edible mushrooms: {edible_count}\nNumber of poisonous mushrooms: {poisonous_count}")

Number of edible mushrooms: 27181
Number of poisonous mushrooms: 33888


In [9]:
import pandas as pd
from UDFs import DecisionTreeClassifier, train_test_partition, compute_error, accuracy_metric

# Load the data
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')

# Encode the target variable as 0-1
mushrooms['class'] = mushrooms['class'].map({'e': 0, 'p': 1})

# Drop the missing values
#mushrooms_1 = mushrooms.dropna(axis=1)

# Do not drop the mising values
mushrooms_1 = mushrooms.copy()

# Create X and y
X = mushrooms_1.drop('class', axis=1).values
y = mushrooms_1['class'].values

# Partition X and y into training and test sets
X_train, X_test, y_train, y_test = train_test_partition(X, y, test_size=0.3, random_state=42)

# Train the classifier (on the training set)
tree = DecisionTreeClassifier(max_depth=10, criterion="gini", n_features="log2", n_quantiles=10)
tree.fit(X_train, y_train)

# Create predictions on the training set
y_train_predicted = tree.predict(X_train)

# Compute the training error according to the zero-one loss
training_error = compute_error(y_train, y_train_predicted)
print(f"Training error: {training_error:.2f}")

# Create predictions on the test set
y_test_predicted = tree.predict(X_test)

# Compute the test error according to the zero-one loss
test_error = compute_error(y_test, y_test_predicted)
print(f"Test error: {test_error:.2f}")

# Compute the accuracy
accuracy = accuracy_metric(y_test, y_test_predicted)
print(f"Accuracy (test set): {accuracy:.2f}")

Fitting the model...
Considering 4 features at each split.
Considering thresholds for feature index 17: ['g' 'k' 'n' 'p' 'r' 'u' 'w']
Considering thresholds for feature index 9: [4.796 9.062000000000001 13.328000000000001 17.594 21.87
 26.146000000000004 30.682000000000002 35.668 46.23800000000001]
Considering thresholds for feature index 18: ['d' 'g' 'h' 'l' 'm' 'p' 'u' 'w']
Considering thresholds for feature index 12: ['b' 'e' 'f' 'g' 'k' 'l' 'n' 'o' 'p' 'r' 'u' 'w' 'y']
Best feature index: 9
Best threshold value: 9.062000000000001
Considering thresholds for feature index 14: ['k' 'n' 'w']
Considering thresholds for feature index 1: ['b' 'c' 'f' 'o' 'p' 's' 'x']
Considering thresholds for feature index 11: ['f' 'g' 'h' 'i' 'k' 's' 't' 'y']
Considering thresholds for feature index 17: ['k' 'n' 'p' 'r' 'u' 'w']
Best feature index: 11
Best threshold value: k
Considering thresholds for feature index 2: ['d' 'e' 'g' 'h' 'i' 'k' 'l' 's' 't' 'w' 'y']
Considering thresholds for feature index

In [7]:
import pandas as pd
import numpy as np
from UDFs import DecisionTreeClassifier, k_fold_partition, compute_error, accuracy_metric

# Load the data
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')

# Encode the target variable as 0-1
mushrooms['class'] = mushrooms['class'].map({'e': 0, 'p': 1})

# Drop the missing values
#mushrooms_1 = mushrooms.dropna(axis=1)

# Do not drop the mising values
mushrooms_1 = mushrooms.copy()

# Create X and y
X = mushrooms_1.drop('class', axis=1).values
y = mushrooms_1['class'].values

## External Cross-Validation ##

# Initialize the classifier
tree = DecisionTreeClassifier(max_depth=10, criterion="gini", n_features="log2", n_quantiles=10)

# Partition X and y into k folds
fold_indices = k_fold_partition(X, random_state=42)

# Initialize a list with test errors
test_errors = []

# Initialize a list with accuracies
accuracies = []

# Perform cross-validation and compute the c-v estimate
for train_indices, test_indices in fold_indices:
    
    X_train, y_train = X[train_indices], y[train_indices]
    X_test, y_test = X[test_indices], y[test_indices]
    
    tree.fit(X_train, y_train)
    
    y_test_predicted = tree.predict(X_test)
    
    test_error = compute_error(y_test, y_test_predicted)
    test_errors.append(test_error)
    
    accuracy = accuracy_metric(y_test, y_test_predicted)
    accuracies.append(accuracy)

mean_test_error = np.mean(test_errors)
mean_accuracy = np.mean(accuracies)

print("K-Fold Cross-Validation Test Errors:", test_errors)
print("Mean Test Error:", mean_test_error)

print("K-Fold Cross-Validation Accuracies:", accuracies)
print("Mean Accuracy:", mean_accuracy)

Fitting the model...
Considering 4 features at each split.
Considering thresholds for feature index 17: ['g' 'k' 'n' 'p' 'r' 'u' 'w']
Considering thresholds for feature index 9: [4.9510000000000005 9.362 13.773000000000001 18.184 22.615000000000002
 27.026000000000003 31.657000000000004 36.936 49.63300000000001]
Considering thresholds for feature index 18: ['d' 'g' 'h' 'l' 'm' 'p' 'u' 'w']
Considering thresholds for feature index 12: ['b' 'e' 'f' 'g' 'k' 'l' 'n' 'o' 'p' 'r' 'u' 'w' 'y']
Best feature index: 9
Best threshold value: 9.362
Considering thresholds for feature index 14: ['k' 'n' 'w']
Considering thresholds for feature index 9: [1.422 2.304 3.1860000000000004 4.0680000000000005 4.95 5.832000000000001
 6.714 7.596 8.478000000000002]
Considering thresholds for feature index 18: ['d' 'g' 'h' 'l' 'm' 'p']
Considering thresholds for feature index 17: ['k' 'n' 'p' 'r' 'u' 'w']
Best feature index: 9
Best threshold value: 3.1860000000000004
Considering thresholds for feature index 11:

In [1]:
import pandas as pd
from UDFs import DecisionTreeClassifier, k_fold_cv_estimate

# Load the data
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')

# Encode the target variable as 0-1
mushrooms['class'] = mushrooms['class'].map({'e': 0, 'p': 1})

# Drop the missing values
#mushrooms_1 = mushrooms.dropna(axis=1)

# Do not drop the mising values
mushrooms_1 = mushrooms.copy()

# Create X and y
X = mushrooms_1.drop('class', axis=1).values
y = mushrooms_1['class'].values

## External Cross-Validation using a Function ##

# Set the parameters and initialize the classifier
parameters = {
    'max_depth': 10,
    'n_features': "log2",
    'criterion': "gini",
    'n_quantiles': 10
    }

tree = DecisionTreeClassifier(**parameters)

# Perform cross-validation and compute the c-v estimate
mean_test_error = k_fold_cv_estimate(X, y, tree, random_state=42)
print("Mean Test Error:", mean_test_error)

Fitting the model...
Considering 4 features at each split.
Considering thresholds for feature index 17: ['g' 'k' 'n' 'p' 'r' 'u' 'w']
Considering thresholds for feature index 9: [4.9510000000000005 9.362 13.773000000000001 18.184 22.615000000000002
 27.026000000000003 31.657000000000004 36.936 49.63300000000001]
Considering thresholds for feature index 18: ['d' 'g' 'h' 'l' 'm' 'p' 'u' 'w']
Considering thresholds for feature index 12: ['b' 'e' 'f' 'g' 'k' 'l' 'n' 'o' 'p' 'r' 'u' 'w' 'y']
Best feature index: 9
Best threshold value: 9.362
Considering thresholds for feature index 14: ['k' 'n' 'w']
Considering thresholds for feature index 9: [1.422 2.304 3.1860000000000004 4.0680000000000005 4.95 5.832000000000001
 6.714 7.596 8.478000000000002]
Considering thresholds for feature index 18: ['d' 'g' 'h' 'l' 'm' 'p']
Considering thresholds for feature index 17: ['k' 'n' 'p' 'r' 'u' 'w']
Best feature index: 9
Best threshold value: 3.1860000000000004
Considering thresholds for feature index 11:

In [6]:
import pandas as pd
from UDFs import train_test_partition, hyperparameter_tuning

# Load the data
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')

# Encode the target variable as 0-1
mushrooms['class'] = mushrooms['class'].map({'e': 0, 'p': 1})

# Drop the missing values
#mushrooms_1 = mushrooms.dropna(axis=1)

# Do not drop the mising values
mushrooms_1 = mushrooms.copy()

# Create X and y
X = mushrooms_1.drop('class', axis=1).values
y = mushrooms_1['class'].values

## Training+Test and Internal Cross-Validation ##

# Partition X and y into training and test sets
X_train, X_test, y_train, y_test = train_test_partition(X, y, test_size=0.3, random_state=42)

# Perform cross-validation to find the best hyperparameter values
parameter_grid = {
    'min_samples_split': [2],
    'max_depth': [5],
    'n_features': [3],
    'criterion': ['entropy'],
    'min_information_gain': [0.0],
    'n_quantiles': [2, 5],
    'isolate_one': [False]
}

best_parameters, best_mean_test_error = hyperparameter_tuning(X_train, y_train, parameter_grid, random_state=42)
print("Best hyperparameters:", best_parameters)
print("Best k-fold cross-validation score:", best_mean_test_error)

Fitting the model...
Considering 3 features at each split.
Considering thresholds for feature index 2: ['d' 'e' 'g' 'h' 'i' 'k' 'l' 's' 't' 'w' 'y']
Considering thresholds for feature index 19: ['a' 's' 'u' 'w']
Considering thresholds for feature index 5: ['a' 'd' 'e' 'f' 'p' 's' 'x']
Best feature index: 5
Best threshold value: d
Considering thresholds for feature index 6: ['c' 'd']
Considering thresholds for feature index 2: ['d' 'e' 'g' 'h' 'i' 'k' 'l' 's' 't' 'w' 'y']
Considering thresholds for feature index 5: ['a' 'd']
Best feature index: 6
Best threshold value: c
Considering thresholds for feature index 1: ['b' 'c' 'f' 'o' 'p' 's' 'x']
Considering thresholds for feature index 0: [8.614999999999998]
Considering thresholds for feature index 9: [13.77]
Best feature index: 9
Best threshold value: 13.77
Considering thresholds for feature index 6: ['c']
Considering thresholds for feature index 10: ['b' 'f' 's']
Considering thresholds for feature index 15: ['f' 't']
Best feature index: 